In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Cargar los datos desde un archivo CSV o cualquier otra fuente
data = pd.read_csv('dataset.csv')

# Seleccionar las columnas relevantes para el modelo de regresión
selected_columns = [ 'Production budget $', 'Worldwide gross $']
data = data[selected_columns]

# Eliminar filas con datos faltantes, si es necesario
data.dropna(inplace=True)

# Dividir los datos en variables predictoras (X) y variable objetivo (y)
X = data.drop('Worldwide gross $', axis=1)
y = data['Worldwide gross $']

# Normalizar las variables predictoras para mejorar la convergencia del modelo
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Aplicar validación cruzada para evaluar el rendimiento del modelo
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
r_squared_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Construir el modelo de regresión utilizando TensorFlow
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(1)
    ])

    # Compilar el modelo
    model.compile(optimizer='adam', loss='mse')

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test).flatten()

    # Evaluar el rendimiento del modelo en esta división
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)

    mse_scores.append(mse)
    r_squared_scores.append(r_squared)

    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r_squared}')
    print()

# Evaluar el rendimiento promedio del modelo a través de todas las divisiones
print(f'Mean MSE: {np.mean(mse_scores)}')
print(f'Mean R-squared: {np.mean(r_squared_scores)}')

28/28 [==============================] - 0s 1ms/step
Mean Squared Error: 6.041849895861244e+16
R-squared: -0.34453537855519367

28/28 [==============================] - 0s 1ms/step
Mean Squared Error: 4.771583912609927e+16
R-squared: -0.33182089456586183

28/28 [==============================] - 0s 1ms/step
Mean Squared Error: 5.88117095940661e+16
R-squared: -0.22506685328161757

